# Creating and using API's in Python

## Introduction
An API allows two pieces of software talk to each other. Just like a function, we don’t have to know how the API works only its inputs and outputs. An essential type of API is a REST API that allows us to access resources via the internet. Reviewing the Pandas Library in the context of an API, and also reviewing a basic REST API.

In [19]:
!pip install pycoingecko
!pip install mplfinance

In [20]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

In [12]:
dict_={'a':[11,21,31],'b':[12,22,32]}

In [14]:
df = pd.DataFrame(dict_)
type(df)

pandas.core.frame.DataFrame

In [15]:
df.head()

,a,b
0,11,12
1,21,22
2,31,32


In [16]:
df.mean()

a    21.0
b    22.0
dtype: float64

Rest API’s function by sending a request, the request is communicated via HTTP message. The HTTP message usually contains a JSON file. This contains instructions for what operation we would like the service or resource to perform. In a similar manner, API returns a response, via an HTTP message, this response is usually contained within a JSON.

In cryptocurrency a popular method to display the movements of the price of a currency is **Candle Stick graph**.

Using the CoinGecko API, we will create one of these candlestick graphs for Bitcoin. We will use the API to get the price data for 30 days with 24 observation per day, 1 per hour. We will find the max, min, open, and close price per day meaning we will have 30 candlesticks and use that to generate the candlestick graph. Although we are using the CoinGecko API we will use a Python client/wrapper for the API called PyCoinGecko. PyCoinGecko will make performing the requests easy and it will deal with the endpoint targeting.

Lets start off by getting the data we need. Using the get_coin_market_chart_by_id(id, vs_currency, days). id is the name of the coin you want, vs_currency is the currency you want the price in, and days is how many days back from today you want.

In [38]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id="bitcoin", vs_currency='inr', days=30)

In [39]:
type(bitcoin_data)

dict

The response (bitcoin_data) we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.

In [40]:
bitcoin_price_data = bitcoin_data["prices"]

In [41]:
bitcoin_price_data[0:5]

[[1618611063218, 4611309.915325079],
 [1618614357415, 4592620.644946132],
 [1618618000894, 4581285.608637168],
 [1618621423176, 4583694.867490095],
 [1618625292971, 4621004.1958030015]]

Finally lets turn this data into a Pandas DataFrame.

In [42]:
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])

In [43]:
data.head()

,TimeStamp,Price
0,1618611063218,4.611310e+06
1,1618614357415,4.592621e+06
2,1618618000894,4.581286e+06
3,1618621423176,4.583695e+06
4,1618625292971,4.621004e+06


Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called Date. We will map our unix_to_datetime to each timestamp and convert it to a readable datetime.

In [44]:
data['Date'] = pd.to_datetime(data['TimeStamp'], unit='ms')

In [46]:
data.tail()

,TimeStamp,Price,Date
717,1621188057420,3.454684e+06,2021-05-16 18:00:57.420
718,1621191718477,3.368788e+06,2021-05-16 19:01:58.477
719,1621195318716,3.315207e+06,2021-05-16 20:01:58.716
720,1621198919957,3.260245e+06,2021-05-16 21:01:59.957
721,1621201323000,3.321152e+06,2021-05-16 21:42:03.000


Using this modified dataset we can now group by the Date and find the min, max, open, and close for the candlesticks.

In [47]:
candlestick_data = data.groupby(data.Date.dt.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

Finally use plotly to create the Candlestick Chart.

In [49]:
fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

## THANK YOU